In [6]:
import stanza
import spacy
import fastcoref
from fastcoref import FCoref
import spacy
from spacy.matcher import Matcher

In [7]:
stanza.download("en")

2025-03-22 10:16:34 INFO: Downloaded file to C:\Users\lbeln\stanza_resources\resources.json
03/22/2025 10:16:34 - INFO - 	 Downloaded file to C:\Users\lbeln\stanza_resources\resources.json
2025-03-22 10:16:34 INFO: Downloading default packages for language: en (English) ...
03/22/2025 10:16:34 - INFO - 	 Downloading default packages for language: en (English) ...
2025-03-22 10:16:37 INFO: File exists: C:\Users\lbeln\stanza_resources\en\default.zip
03/22/2025 10:16:37 - INFO - 	 File exists: C:\Users\lbeln\stanza_resources\en\default.zip
2025-03-22 10:16:46 INFO: Finished downloading models and saved to C:\Users\lbeln\stanza_resources
03/22/2025 10:16:46 - INFO - 	 Finished downloading models and saved to C:\Users\lbeln\stanza_resources


In [8]:
nlp = stanza.Pipeline('en')

2025-03-22 10:16:49 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
03/22/2025 10:16:49 - INFO - 	 Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-03-22 10:16:49 INFO: Downloaded file to C:\Users\lbeln\stanza_resources\resources.json
03/22/2025 10:16:49 - INFO - 	 Downloaded file to C:\Users\lbeln\stanza_resources\resources.json
2025-03-22 10:16:52 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

03/22/2025 10:16:52 - INFO - 	 Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined

In [9]:
def find_dependents(id, doc):
    dependents = []
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.head == id:
                dependents.append(word)
    return dependents

def find_start_verb(doc):
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.upos == "VERB":
                return word
    return None

def find_object(doc, verb):
    dependents = find_dependents(verb.id, doc)
    for word in dependents:
        if word.deprel == "obj" or word.deprel == "nsubj:pass":
            return word
    return None

def find_subject(doc, verb):
    dependents = find_dependents(verb.id, doc)
    for word in dependents:
        if word.deprel == "nsubj" or word.deprel == "obl:agent":
            return word
    
    for word in dependents:
        if word.deprel == "advcl":
            subject = find_subject(doc, word)
            if subject:
                return subject

    return None

In [10]:
def find_noun(doc, word):
    dependents = find_dependents(word.id, doc)
    for dependent in dependents:
        if dependent.deprel == "nmod:poss":
            return find_noun(doc, dependent)
    if word.upos == "NOUN" or word.upos == "PRON":
        return word
    return None

In [11]:
def same_reference(sentence, a, b, doc=None):
    if not doc:
        doc = nlp(sentence)

    model = FCoref(enable_progress_bar=False)
    
    noun_a = find_noun(doc, a)
    noun_b = find_noun(doc, b)

    if not noun_a or not noun_b:
        return False

    a_pos = (noun_a.start_char, noun_a.end_char)
    b_pos = (noun_b.start_char, noun_b.end_char)

    a_found = False
    b_found = False
    
    clusters = model.predict(texts=[sentence])[0].get_clusters(as_strings=False)
    for cluster in clusters:
        for c in cluster:
            if a_pos[0] >= c[0] and a_pos[0] <= c[1] and a_pos[1] >= c[0] and a_pos[1] <= c[1]:
                a_found = True
            if b_pos[0] >= c[0] and b_pos[0] <= c[1] and b_pos[1] >= c[0] and b_pos[1] <= c[1]:
                b_found = True
            if a_found and b_found:
                return True

    return False

In [12]:
def find_relations(sentence):
    relations = []
    doc = nlp(sentence)
    for sent in doc.sentences:
        for word in sent.words:
            if word.upos != "VERB":
                continue
            verb = word
            object = find_object(doc, verb)
            subject = find_subject(doc, verb)
            if verb and object and subject and not same_reference(sentence, object, subject, doc=doc) and is_species_or_trait(sentence, object) and is_species_or_trait(sentence, subject):
                relations.append({"verb": verb, "object": object, "subject": subject})
    return relations

In [13]:
def find_context(sentence, stanza_word):
    snlp = spacy.load("en_core_web_sm")
    matcher = Matcher(snlp.vocab)
    
    pattern = [
        {"POS": {"IN": ["DET", "ADJ", "NOUN", "PROPN"]}, "OP": "+"},
        {"POS": "ADP", "OP": "?"},
        {"POS": {"IN": ["DET", "ADJ", "NOUN", "PROPN"]}, "OP": "+"}
    ]
    matcher.add("NOUN_PHRASE", [pattern])

    doc = snlp(sentence)
    spans = [doc[start:end] for _, start, end in matcher(doc)]
    for span in spacy.util.filter_spans(spans):
        for word in span:
            if stanza_word.start_char == word.idx and stanza_word.end_char == (word.idx + len(word)):
                return span

    return None

In [14]:
def find_species_or_trait(context_span):
    contains_traits = ""
    contains_species = ""

    # There is no current way to determine whether
    # a species is included or not, I'd have to fine-tune
    # a model for that. So, for now, I'm hard-coding it.
    for word in context_span:
        if word.text[0:2] == "TR":
            contains_traits = "TR"
        if word.text[0:2] == "SP":
            contains_species = "SP"

    return [contains_traits, contains_species]

In [15]:
def is_species_or_trait(sentence, stanza_word):
    context = find_context(sentence, stanza_word)
    if not context:
        return False
    
    st_context = find_species_or_trait(context)
    return st_context[0] != "" or st_context[1] != ""

In [46]:
sentence_basic = "Presence of SPECIES1 reduced the TRAIT of SPECIES2 on SPECIES3"
print(f"Sentence: \"{sentence_basic}\"")
relations = find_relations(sentence_basic)

if not relations:
    print("No Relations")
for r in relations:
    print(f"Subject: \"{r['subject'].text}\"")
    print(f"Subject Context: \"{find_context(sentence_basic, r['subject'])}\"\n")
    print(f"Verb: \"{r['verb'].text}\"\n")
    print(f"Object: \"{r['object'].text}\"")
    print(f"Object Context: \"{find_context(sentence_basic, r['object'])}\"")

Sentence: "Presence of SPECIES1 reduced the TRAIT of SPECIES2 on SPECIES3"


03/14/2025 18:25:17 - INFO - 	 missing_keys: []
03/14/2025 18:25:17 - INFO - 	 unexpected_keys: []
03/14/2025 18:25:17 - INFO - 	 mismatched_keys: []
03/14/2025 18:25:17 - INFO - 	 error_msgs: []
03/14/2025 18:25:17 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
03/14/2025 18:25:17 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

03/14/2025 18:25:17 - INFO - 	 ***** Running Inference on 1 texts *****


Subject: "Presence"
Subject Context: "Presence of SPECIES1"

Verb: "reduced"

Object: "TRAIT"
Object Context: "the TRAIT of SPECIES2"


In [48]:
sentence_basic = "Bob reduced Joe's level of stress."
print(f"Sentence: \"{sentence_basic}\"")
relations = find_relations(sentence_basic)

if not relations:
    print("No Relations")
for r in relations:
    print(f"Subject: \"{r['subject'].text}\"")
    print(f"Subject Context: \"{find_context(sentence_basic, r['subject'])}\"\n")
    print(f"Verb: \"{r['verb'].text}\"\n")
    print(f"Object: \"{r['object'].text}\"")
    print(f"Object Context: \"{find_context(sentence_basic, r['object'])}\"")

Sentence: "Bob reduced Joe's level of stress."


03/14/2025 19:04:35 - INFO - 	 missing_keys: []
03/14/2025 19:04:35 - INFO - 	 unexpected_keys: []
03/14/2025 19:04:35 - INFO - 	 mismatched_keys: []
03/14/2025 19:04:35 - INFO - 	 error_msgs: []
03/14/2025 19:04:35 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M


No Relations


In [2]:
abstracts = [
    "Many prey modify traits in response to predation risk and this modification of traits can influence the prey's resource acquisition rate. A predator thus can have a â€œnonlethalâ€ impact on prey that can lead to indirect effects on other community members. Such indirect interactions are termed trait-mediated indirect interactions because they arise from a predator's influence on prey traits, rather than prey density. Because such nonlethal predator effects are immediate, can influence the entire prey population, and can occur over the entire prey lifetime, we argue that nonlethal predator effects are likely to contribute strongly to the net indirect effects of predators (i.e., nonlethal effects may be comparable in magnitude to those resulting from killing prey). This prediction was supported by an experiment in which the indirect effects of a larval dragonfly (Anax sp.) predator on large bullfrog tadpoles (Rana catesbeiana), through nonlethal effects on competing small bullfrog tadpoles, were large relative to indirect effects caused by density reduction of the small tadpoles (the lethal effect). Treatments in which lethal and nonlethal effects of Anax were manipulated independently indicated that this result was robust for a large range of different combinations of lethal and nonlethal effects. Because many, if not most, prey modify traits in response to predators, our results suggest that the magnitude of interaction coefficients between two species may often be dynamically related to changes in other community members, and that many indirect effects previously attributed to the lethal effects of predators may instead be due to shifts in traits of surviving prey.",
    "With global change modifying species assemblages, our success in predicting ecosystem level consequences of these new communities will depend, in part, on our ability to understand biotic interactions. Current food web theory considers interactions between numerous species simultaneously, but descriptive models are unable to predict interactions between newly co-occurring species. Incorporating proxies such as functional traits and phylogeny into models could help infer predator/ prey interactions. Here we used trait-matching between predator feeding traits and prey vulnerability traits, along with phylogeny (used as a proxy for chemical defence and other traits difficult to document), to infer predatory interactions using ground beetles as model organisms. A feeding experiment was conducted involving 20 ground beetle and 115 prey species to determine which pair of species did or did not interact. Eight predator and four prey functional traits were measured directly on specimens. Then, using a modeling approach based on the matching-centrality formalism, we evaluated 511 predictive ecological models that tested different combinations of all predator and prey functional traits, and phylogenetic information. The most parsimonious model accurately predicted 81 % of the observed realized and unrealized interactions, using phylogenetic information and the trait-matches predator biting force/ prey cuticular toughness and predator/ prey body size ratio. The best trait-based models predicted correctly >80 % which species interact (realized interactions), but predict 75 %. The matching of predator biting force and prey cuticular toughness demonstrated a better predictive power than the commonly used predator/ prey body size ratio. Our novel model combining both functional traits and phylogeny extends beyond existing descriptive approaches and could represent a valuable tool to predict consumer/ resource interactions of newly introduced species and to resolve cryptic food webs. This article is protected by copyright. All rights reserved.",
    "Only a fraction of the individuals in a given prey population are likely to be killed and consumed by predators. In contrast, nearly all individuals experience the chronic effects of predation risk. When threatened by predators, prey adopt defensive tactics whose costs can lead to reduced growth, maturation rates, survivorship, fecundity, or pop- ulation density. This nonconsumptive impact of predation risk on prey is known as a ''trait- mediated interaction'' (TMI) because it results from changes in prey traits such as behavior or physiology. Ecological theory suggests that the strength of TMI effects will reflect a balance between the conflicting demands of reproduction vs. predator avoidance. Competitor density and resource availability are expected to alter the balance between these conflicting forces. We conducted a meta-analysis of experimental studies that measured TMI effect size while varying competitor and/or resource density. The threat of predation had an overall negative effect on prey performance, but the strength of this effect varied with the level of competition. High competition exacerbated the negative effect of intimidation on prey density but moderated the negative effect of intimidation on prey life history and growth. We discuss these results in light of previously published theoretical expectations. Our results highlight the variable and context-dependent nature of interspecific interactions.",
    "Trait-mediated interactions (TMII) can alter the outcome or magnitude of species interactions. We examined how the interaction between a guild of ground and rove beetles and their fly egg prey was altered by a larger predator, the ground beetle Pterostichus melanarius, and an additional prey, aphids. In field and laboratory experiments, we manipulated the presence or absence of P. melanarius and aphids and recorded the impact of these manipulations on beetle activity and fly egg predation. Individually, aphids, by serving as preferred prey, and P. melanarius, by reducing focal beetle activity, weakened egg predation. However, egg predation was restored when both aphids and P. melanarius were present together, because aphids triggered greater foraging activity, and thus increased incidental predation of fly eggs, by P. melanarius. Thus, TMII among subsets of the community that were disruptive to predation on fly eggs could not be summed to predict the dominant, positive TMII within a more diverse community. Future TMII studies should include more realistic representations of species diversity, and should not ignore the influence of prey on predator behavior.",
    "This investigation examines the role of trait-mediated indirect interactions in a simple aquatic food web. We conducted the experiments in cattle watering tanks in order to establish whether competitive and predator-prey interactions between two species are affected by other species in the system; i.e., are pairwise interaction strengths affected by the background species assemblage? We examined the survival and growth response of small bullfrog (Rana catesbeiana) and small green frog (Rana clamitans) tadpoles in the presence and absence of a competitor (large bullfrogs), the lethal presence of the larval odonate predator Tramea lacerata,and the nonlethal (caged) presence of the larval odonate predators Anax junius and Anax longipes. We demonstrate that large bullfrog competitors and caged Anax affect traits (foraging activity level) of small bullfrog and small green frog tadpoles and that these changes in traits, in turn, affect interactions of the small tadpole species with each other and with the other species. In particular, the following four trait- mediated indirect interactions were evident: (1) Presence of large bullfrog competitors increased the predation rate of Trameaon small green frogs and small bullfrogs. (2) Presence of nonlethal Anax reduced the predation rate of Tramea on small green frogs. (3) Presence of nonlethal Anax increased the competitive advantage of bullfrogs over green frogs. (4) Presence of nonlethal Anax facilitated midge invasion of the experimental units. The pro- posed mechanisms (changes in small tadpole activity) involved in these trait-mediated indirect interactions were supported by observational data on tadpole activity and resource levels in the experimental units, and in laboratory experiments examining tadpole activity responses to predators. The occurrence of strong trait-mediated indirect interactions in this simple food web underscores the potential importance of such interactions in animal communities.",
    "Trait-mediated interactions (TMIs), in which trophic and competitive inter- actions depend on individual traits as well as on overall population densities, have inspired large amounts of research, but theoretical and empirical studies have not been well con- nected. To help mitigate this problem, we review and synthesize the theoretical literature on TMIs and, in particular, on trait-mediated indirect interactions, TMIIs, in which the presence of one species mediates the interaction between a second and third species. (1) In models, TMIs tend to stabilize simple communities; adding further biological detail often reduces stability in models, but populations may persist even if their dynamics become mathematically unstable. (2) Short- and long-term changes in population density caused by TMIs depend even more on details, such as the curvature of functional responses and trade-offs, which have rarely been measured. (3) The effects of TMIs in multipredator communities depend in a straightforward way on the specificity of prey defenses. (4) Tritrophic and more complex communities are theoretically difficult; few general conclu- sions have emerged. Theory needs new kinds of experiments as a guide. The most critical needs are experiments that measure curvatures of trade-offs and responses, and experiments that (combined with theory) allow us to scale from short- to long-term responses of com- munities. Anecdotal evidence from long-term and large-scale studies suggests that TMIs may affect community dynamics at practical management scales; community models in- corporating TMIs are necessary and require closer collaborations between theory and ex-",
    "Non-lethal, trait-mediated effects of predation impact prey behavior and life-history traits. Studying how these effects in turn influence prey demography is crucial to understand prey life-history evolution. Mosquitoes are important vectors that claim several million lives every year worldwide by transmitting a range of pathogens. Several ecological factors affect life-history traits of both larval and adult mosquitoes, creating effects that cascade to population-level consequences. Few studies have comprehensively explored the non-lethal effects of predation and its interactions with resources and competition on larval, adult, and population traits of mosquitoes. Understanding these interactions is important because the effects of predation are hypothesized to rescue prey populations from the effects of density-dependence resulting from larval competition. Aedes aegypti larvae reared at two different larval densities and subjected to three non-lethal predator treatments were monitored for survival, development time, and adult size through the larval stages to adult eclosion, and adult females were monitored for survival and reproduction through their first gonotrophic cycle. Intraspecific competition increased larval development time, yielded small-bodied adults, and reduced fecundity in individuals exposed to predatory chemical cues as larvae. Exposure to cues from a living predator affected both body size and latency to blood feed in females. Analysis of life-table traits revealed significant effects of competition on net reproductive rate (R 0) of mosquitoes. The interaction between competition and predator treatments significantly affected the cohort rate of increase (r) and the index of performance (r'). The index of performance, which estimates rate of population change based on the size-fecundity relationship, was significantly and positively correlated with r, but overestimated r slightly. Lack of significant effect of predator treatments and larval density on cohort generation time (T c) further suggests that the observed effects of treatments on r and r' were largely a consequence of the effects on R 0. Also, the significant effects of treatment combinations on larval development time, adult body size and fecundity were ultimately manifested as effects on life-table traits estimated from adult survival and reproduction.",
    "The outcome of species interactions is often strongly influenced by variation in the functional traits of the individuals participating. A rather large body of work demonstrates that inducible morphological plasticity in predators and prey can both influence and be influenced by species interaction strength, with important consequences for individual fitness. Much of the past research in this area has focused on the ecological and evolutionary significance of trait plasticity by studying single predatorâ€“prey pairs and testing the performance of individuals having induced and noninduced phenotypes. This research has thus been critical in improving our understanding of the adaptive value of trait plasticity and its widespread occurrence across species and community types. More recently, researchers have expanded this foundation by examining how the complexity of organismal design and community-level properties can shape plasticity in functional traits. In addition, researchers have begun to merge evolutionary and ecological perspectives by linking trait plasticity to community dynamics, with particular attention on trait-mediated indirect interactions. Here, we review recent studies on inducible morphological plasticity in predators and their prey with an emphasis on internal and external constraints and how the nature of predatorâ€“prey interactions influences the expression of inducible phenotypes. In particular, we focus on multiple-trait plasticity, flexibility and modification of inducible plasticity, and reciprocal plasticity between predator and prey. Based on our arguments on these issues, we propose future research directions that should better integrate evolutionary and population studies and thus improve our understanding of the role of phenotypic plasticity in predatorâ€“prey population and community dynamics.",
    "This article compares indirect trait-mediated interactions in simple resource-consumer-predator food webs with those that are density-mediated. It focuses on two well documented responses of consumers to predation risk: decrease in consumer activity and habitat switch. These behavioral effects are transmitted to resources and they cause similar indirect effects as those which are mediated by density changes in consumers. Two indirect interactions are studied in this article: trophic cascades, and apparent competition. Results for density only, trait only and combined density and trait mediated interactions are compared and discussed with respect to manipulation with predator density (top-down manipulation) and resource environmental capacity (bottom-up manipulation). The article shows that trait-mediated, effects on species equilibrial densities are similar to those of density-mediated, but they are often highly non-linear. Thus, they may have potential for even stronger impact on food webs than those which are density mediated.",
    "In this paper we review the empirical studies documenting trait-mediated indirect interactions (TMIIs) in food webs. Basic models and empirical approaches that form the foundation of our conceptualization of species interactions generally assume that interactions are an intrinsic property of the two interacting species and therefore are governed by their respective densities. However, if a species reacts to the presence of a second species by altering its phenotype, then the trait changes in the reacting species can alter the per capita effect of the reacting species on other species and, consequently, population density or fitness of the other species. Such trait-mediated indirect interactions can reinforce or oppose density-mediated effects and have been largely overlooked by community ecologists. We first briefly develop the case for the broad mechanistic basis for TMIIs and then review the direct evidence for TMIIs in various permutations of simple three- to four species food webs. We find strong evidence for quantitatively significant effects of TMIIs in a variety of aquatic and terrestrial systems. We further highlight those few studies that address the question of the relative magnitudes of density- and trait-mediated effects and the role of species densities in their transmission. These studies indicate that trait effects are often as strong or stronger than density effects. We conclude that ecologicalcommunities are replete with TMIIs arising from trait plasticity and that these effects are quantitatively important to community dynamics. Finally, we synthesize our results and indicate profitable directions for future research."
    '''Trait-mediated interactions (TMII) can alter the outcome or magnitude of
     species interactions. We examined how the interaction between a guild of ground and rove
     beetles and their fly egg prey was altered by a larger predator, the ground beetle Pterostichus
     melanarius, and an additional prey, aphids. In field and laboratory experiments, we
     manipulated the presence or absence of P. melanarius and aphids and recorded the impact
     of these manipulations on beetle activity and fly egg predation. Individually, aphids, by
     serving as preferred prey, and P. melanarius, by reducing focal beetle activity, weakened egg
     predation. However, egg predation was restored when both aphids and P. melanarius were
     present together, because aphids triggered greater foraging activity, and thus increased
     incidental predation of fly eggs, by P. melanarius. Thus, TMII among subsets of the
     community that were disruptive to predation on fly eggs could not be summed to predict the
     dominant, positive TMII within a more diverse community. Future TMII studies should
     include more realistic representations of species diversity, and should not ignore the influence
     of prey on predator behavior.
    ''',
    ''' Natural enemies not only influence prey density but they can also cause the modification of traits
         in their victims. While such non-lethal effects can be very
         important for the dynamic and structure of prey popula
         tions, little is known about their interaction with the
         density-mediated effects of natural enemies. We investi
         gated the relationship between predation rate, prey
         density and trait modification in two aphid-aphid predator
         interactions. Pea aphids (Acyrthosiphon pisum, Harris)
         have been shown to produce winged dispersal morphs in
         response to the presence of ladybirds or parasitoid natural
         enemies. This trait modification influences the ability of
         aphids to disperse and to colonise new habitats, and hence
         has a bearing on the population dynamics of the prey. In
         two experiments we examined wing induction in pea
         aphids as a function of the rate of predation when
         hoverfly larvae (Episyrphus balteatus) and lacewing
         larvae (Chrysoperla carnea) were allowed to forage in
         pea aphid colonies. Both hoverfly and lacewing larvae
         caused a significant increase in the percentage of winged
         morphs among offspring compared to control treatments,
         emphasising that wing induction in the presence of natural
         enemies is a general response in pea aphids. The
         percentage of winged offspring was, however, dependent
         on the rate of predation, with a small effect of predation
         on aphid wing induction at very high and very low
         predation rates, and a strong response of aphids at
         medium predation rates. Aphid wing induction was
         influenced by the interplay between predation rate and
         the resultant prey density. Our results suggests that
         density-mediated and trait-mediated effects of natural
         enemies are closely connected to each other and jointly
         determine the effect of natural enemies on prey popula
        tion dynamics.
    '''
]

In [17]:
abstracts_basic = [
    "Many prey modify traits in response to predation risk and this modification of traits can influence the prey's resource acquisition rate. A predator thus can have a â€œnonlethalâ€ impact on prey that can lead to indirect effects on other community members. Such indirect interactions are termed trait-mediated indirect interactions because they arise from a predator's influence on prey traits, rather than prey density. Because such nonlethal predator effects are immediate, can influence the entire prey population, and can occur over the entire prey lifetime, we argue that nonlethal predator effects are likely to contribute strongly to the net indirect effects of predators (i.e., nonlethal effects may be comparable in magnitude to those resulting from killing prey). This prediction was supported by an experiment in which the indirect effects of a SPECIES1 predator on SPECIES2, through nonlethal effects on competing SPECIES3, were large relative to indirect effects caused by density reduction of the small tadpoles (the lethal effect). Treatments in which lethal and nonlethal effects of Anax were manipulated independently indicated that this result was robust for a large range of different combinations of lethal and nonlethal effects. Because many, if not most, prey modify traits in response to predators, our results suggest that the magnitude of interaction coefficients between two species may often be dynamically related to changes in other community members, and that many indirect effects previously attributed to the lethal effects of predators may instead be due to shifts in traits of surviving prey.",
    "With global change modifying species assemblages, our success in predicting ecosystem level consequences of these new communities will depend, in part, on our ability to understand biotic interactions. Current food web theory considers interactions between numerous species simultaneously, but descriptive models are unable to predict interactions between newly co-occurring species. Incorporating proxies such as functional traits and phylogeny into models could help infer predator/ prey interactions. Here we used trait-matching between predator feeding traits and prey vulnerability traits, along with phylogeny (used as a proxy for chemical defence and other traits difficult to document), to infer predatory interactions using ground beetles as model organisms. A feeding experiment was conducted involving 20 ground beetle and 115 prey species to determine which pair of species did or did not interact. Eight predator and four prey functional traits were measured directly on specimens. Then, using a modeling approach based on the matching-centrality formalism, we evaluated 511 predictive ecological models that tested different combinations of all predator and prey functional traits, and phylogenetic information. The most parsimonious model accurately predicted 81 % of the observed realized and unrealized interactions, using phylogenetic information and the trait-matches predator biting force/ prey cuticular toughness and predator/ prey body size ratio. The best trait-based models predicted correctly >80 % which species interact (realized interactions), but predict 75 %. The matching of predator biting force and prey cuticular toughness demonstrated a better predictive power than the commonly used predator/ prey body size ratio. Our novel model combining both functional traits and phylogeny extends beyond existing descriptive approaches and could represent a valuable tool to predict consumer/ resource interactions of newly introduced species and to resolve cryptic food webs. This article is protected by copyright. All rights reserved.",
    "Only a fraction of the individuals in a given prey population are likely to be killed and consumed by predators. In contrast, nearly all individuals experience the chronic effects of predation risk. When threatened by predators, prey adopt defensive tactics whose costs can lead to reduced growth, maturation rates, survivorship, fecundity, or pop- ulation density. This nonconsumptive impact of predation risk on prey is known as a ''trait- mediated interaction'' (TMI) because it results from changes in prey traits such as behavior or physiology. Ecological theory suggests that the strength of TMI effects will reflect a balance between the conflicting demands of reproduction vs. predator avoidance. Competitor density and resource availability are expected to alter the balance between these conflicting forces. We conducted a meta-analysis of experimental studies that measured TMI effect size while varying competitor and/or resource density. The threat of predation had an overall negative effect on prey performance, but the strength of this effect varied with the level of competition. High competition exacerbated the negative effect of intimidation on prey density but moderated the negative effect of intimidation on prey life history and growth. We discuss these results in light of previously published theoretical expectations. Our results highlight the variable and context-dependent nature of interspecific interactions.",
    "Trait-mediated interactions (TMII) can alter the outcome or magnitude of species interactions. We examined how the interaction between a guild of ground and rove beetles and their fly egg prey was altered by a larger predator, the ground beetle Pterostichus melanarius, and an additional prey, aphids. In field and laboratory experiments, we manipulated the presence or absence of P. melanarius and aphids and recorded the impact of these manipulations on beetle activity and fly egg predation. Individually, aphids, by serving as preferred prey, and P. melanarius, by reducing focal beetle activity, weakened egg predation. However, egg predation was restored when both aphids and P. melanarius were present together, because aphids triggered greater foraging activity, and thus increased incidental predation of fly eggs, by P. melanarius. Thus, TMII among subsets of the community that were disruptive to predation on fly eggs could not be summed to predict the dominant, positive TMII within a more diverse community. Future TMII studies should include more realistic representations of species diversity, and should not ignore the influence of prey on predator behavior.",
    "This investigation examines the role of trait-mediated indirect interactions in a simple aquatic food web. We conducted the experiments in cattle watering tanks in order to establish whether competitive and predator-prey interactions between two species are affected by other species in the system; i.e., are pairwise interaction strengths affected by the background species assemblage? We examined the survival and growth response of SPECIES_SMALL_BULLFROG and SPECIES_SMALL_GREEN_FROG in the presence and absence of a competitor SPECIES_LARGE_BULLFROG, the lethal presence of the SPECIES_LARVAL_TRAMEA_LACERATA,and the nonlethal (caged) presence of the larval odonate predators SPECIES_LARVAL_ANAX_JUNIUS and SPECIES_LARVAL_ANAX_LONGPIPES. We demonstrate that SPECIES_LARGE_BULLFROG competitors and caged SPECIES_ANAX affect traits (TRAIT) of SPECIES_SMALL_BULLFROG and SPECIES_SMALL_GREEN_FROG and that these changes in traits, in turn, affect interactions of the small tadpole species with each other and with the other species. In particular, the following four trait- mediated indirect interactions were evident: (1) Presence of SPECIES_BULLFROG competitors increased the predation rate of SPECIES_SMALL_GREEN_FROGS and SPECIES_SMALL_BULLFROGS. (2) Presence of nonlethal SPECIES_ANAX reduced the TRAIT of SPECIES_TRAMEA on SPECIES_SMALL_GREEN_FROGS. (3) Presence of nonlethal SPECIES_ANAX increased the TRAIT of SPECIES_BULLFROG over SPECIES_GREEN_FROGS. (4) Presence of nonlethal SPECIES_ANAX facilitated midge invasion of the experimental units. The pro- posed mechanisms (changes in small tadpole activity) involved in these trait-mediated indirect interactions were supported by observational data on tadpole activity and resource levels in the experimental units, and in laboratory experiments examining tadpole activity responses to predators. The occurrence of strong trait-mediated indirect interactions in this simple food web underscores the potential importance of such interactions in animal communities.",
    "Trait-mediated interactions (TMIs), in which trophic and competitive inter- actions depend on individual traits as well as on overall population densities, have inspired large amounts of research, but theoretical and empirical studies have not been well con- nected. To help mitigate this problem, we review and synthesize the theoretical literature on TMIs and, in particular, on trait-mediated indirect interactions, TMIIs, in which the presence of one species mediates the interaction between a second and third species. (1) In models, TMIs tend to stabilize simple communities; adding further biological detail often reduces stability in models, but populations may persist even if their dynamics become mathematically unstable. (2) Short- and long-term changes in population density caused by TMIs depend even more on details, such as the curvature of functional responses and trade-offs, which have rarely been measured. (3) The effects of TMIs in multipredator communities depend in a straightforward way on the specificity of prey defenses. (4) Tritrophic and more complex communities are theoretically difficult; few general conclu- sions have emerged. Theory needs new kinds of experiments as a guide. The most critical needs are experiments that measure curvatures of trade-offs and responses, and experiments that (combined with theory) allow us to scale from short- to long-term responses of com- munities. Anecdotal evidence from long-term and large-scale studies suggests that TMIs may affect community dynamics at practical management scales; community models in- corporating TMIs are necessary and require closer collaborations between theory and ex-",
    # Not incredibly sure, from this abstract, whether this would be related to the TAH or not.
    "Non-lethal, trait-mediated effects of predation impact prey behavior and life-history traits. Studying how these effects in turn influence prey demography is crucial to understand prey life-history evolution. SPECIES_MOSQUITOES are important vectors that claim several million lives every year worldwide by transmitting a range of pathogens. Several ecological factors affect life-history traits of both larval and adult SPECIES_MOSQUITOES, creating effects that cascade to population-level consequences. Few studies have comprehensively explored the non-lethal effects of predation and its interactions with resources and competition on larval, adult, and population traits of SPECIES_MOSQUITOES. Understanding these interactions is important because the effects of predation are hypothesized to rescue prey populations from the effects of density-dependence resulting from larval competition. SPECIES_LARVAE reared at two different TRAIT_LEVELS and subjected to three non-lethal predator treatments were monitored for survival, development time, and adult size through the larval stages to adult eclosion, and adult females were monitored for survival and reproduction through their first gonotrophic cycle. Intraspecific competition increased TRAIT1, yielded TRAIT2, and reduced TRAIT3 in individuals exposed to predatory chemical cues as SPECIES_LARVAE. Exposure to cues from a living predator affected both TRAIT4 and TRAIT5 in females. Analysis of life-table traits revealed significant effects of competition on TRAIT7 (R 0) of SPECIES_MOSQUITOES. The interaction between competition and predator treatments significantly affected the cohort rate of increase (r) and the index of performance (r'). The index of performance, which estimates rate of population change based on the size-fecundity relationship, was significantly and positively correlated with r, but overestimated r slightly. Lack of significant effect of predator treatments and larval density on cohort generation time (T c) further suggests that the observed effects of treatments on r and r' were largely a consequence of the effects on R 0. Also, the significant effects of treatment combinations on larval development time, adult body size and fecundity were ultimately manifested as effects on life-table traits estimated from adult survival and reproduction.",
    "The outcome of species interactions is often strongly influenced by variation in the functional traits of the individuals participating. A rather large body of work demonstrates that inducible morphological plasticity in predators and prey can both influence and be influenced by species interaction strength, with important consequences for individual fitness. Much of the past research in this area has focused on the ecological and evolutionary significance of trait plasticity by studying single predatorâ€“prey pairs and testing the performance of individuals having induced and noninduced phenotypes. This research has thus been critical in improving our understanding of the adaptive value of trait plasticity and its widespread occurrence across species and community types. More recently, researchers have expanded this foundation by examining how the complexity of organismal design and community-level properties can shape plasticity in functional traits. In addition, researchers have begun to merge evolutionary and ecological perspectives by linking trait plasticity to community dynamics, with particular attention on trait-mediated indirect interactions. Here, we review recent studies on inducible morphological plasticity in predators and their prey with an emphasis on internal and external constraints and how the nature of predatorâ€“prey interactions influences the expression of inducible phenotypes. In particular, we focus on multiple-trait plasticity, flexibility and modification of inducible plasticity, and reciprocal plasticity between predator and prey. Based on our arguments on these issues, we propose future research directions that should better integrate evolutionary and population studies and thus improve our understanding of the role of phenotypic plasticity in predatorâ€“prey population and community dynamics.",
    "This article compares indirect trait-mediated interactions in simple resource-consumer-predator food webs with those that are density-mediated. It focuses on two well documented responses of consumers to predation risk: decrease in consumer activity and habitat switch. These behavioral effects are transmitted to resources and they cause similar indirect effects as those which are mediated by density changes in consumers. Two indirect interactions are studied in this article: trophic cascades, and apparent competition. Results for density only, trait only and combined density and trait mediated interactions are compared and discussed with respect to manipulation with predator density (top-down manipulation) and resource environmental capacity (bottom-up manipulation). The article shows that trait-mediated, effects on species equilibrial densities are similar to those of density-mediated, but they are often highly non-linear. Thus, they may have potential for even stronger impact on food webs than those which are density mediated.",
    "In this paper we review the empirical studies documenting trait-mediated indirect interactions (TMIIs) in food webs. Basic models and empirical approaches that form the foundation of our conceptualization of species interactions generally assume that interactions are an intrinsic property of the two interacting species and therefore are governed by their respective densities. However, if a species reacts to the presence of a second species by altering its phenotype, then the trait changes in the reacting species can alter the per capita effect of the reacting species on other species and, consequently, population density or fitness of the other species. Such trait-mediated indirect interactions can reinforce or oppose density-mediated effects and have been largely overlooked by community ecologists. We first briefly develop the case for the broad mechanistic basis for TMIIs and then review the direct evidence for TMIIs in various permutations of simple three- to four species food webs. We find strong evidence for quantitatively significant effects of TMIIs in a variety of aquatic and terrestrial systems. We further highlight those few studies that address the question of the relative magnitudes of density- and trait-mediated effects and the role of species densities in their transmission. These studies indicate that trait effects are often as strong or stronger than density effects. We conclude that ecologicalcommunities are replete with TMIIs arising from trait plasticity and that these effects are quantitatively important to community dynamics. Finally, we synthesize our results and indicate profitable directions for future research."
    '''Trait-mediated interactions (TMII) can alter the outcome or magnitude of
     species interactions. We examined how the interaction between a guild of ground and SPECIES1 and their fly SPECIES2 was altered by a larger predator, the ground beetle SPECIESPMEL, and an additional prey, aphids. In field and laboratory experiments, we
     manipulated the presence or absence of SPECIESPMEL and SPECIESAPHIDS and recorded the impact
     of these manipulations on TRAIT and TRAIT2. Individually, SPECIESAPHIDS, by
     serving as preferred prey, and SPECIESPMEL, by reducing focal TRAIT1, weakened TRAIT2. However, TRAIT1 was restored when both SPECIESAPHIDS and SPECIESPMEL were
     present together, because SPECIESAPHIDS triggered greater TRAIT, and thus increased
     incidental TRAIT of SPECIESFLYEGGS, by SPECIESPMEL. Thus, TMII among subsets of the
     community that were disruptive to TRAIT on SPECIESFLYEGGS could not be summed to predict the
     dominant, positive TMII within a more diverse community. Future TMII studies should
     include more realistic representations of species diversity, and should not ignore the influence
     of prey on predator behavior.
    ''',
    ''' Natural enemies not only influence prey density but they can also cause the modification of traits
         in their victims. While such non-lethal effects can be very
         important for the dynamic and structure of prey popula
         tions, little is known about their interaction with the
         density-mediated effects of natural enemies. We investi
         gated the relationship between predation rate, prey
         density and trait modification in two aphid-aphid predator
         interactions. SPECIESAPHIDS
         have been shown to produce TRAITS in
         response to the presence of SPECIESLADYBIRDS or SPECIESENIMIES. This trait modification influences the ability of
         aphids to disperse and to colonise new habitats, and hence
         has a bearing on the population dynamics of the prey. In
         two experiments we examined TRAIT in SPECIESAPHIDS as a function of the TRAIT when
         SPECIESHOVERFLY and SPECIESLACEWING were allowed to forage in
         SPECIESAPHIDS colonies. Both SPECIESHOVERFLY and SPECIESLACEWING
         caused a significant increase in the percentage of TRAIT among offspring compared to control treatments,
         emphasising that wing induction in the presence of natural
         enemies is a general response in SPECIESAPHIDS. The
         percentage of winged offspring was, however, dependent
         on the TRAIT, with a small effect of predation
         on SPECIESAPHID TRAIT at very high and very low
         predation rates, and a strong response of SPECIESAPHIDS at
         medium predation rates. Aphid wing induction was
         influenced by the interplay between predation rate and
         the resultant prey density. Our results suggests that
         density-mediated and trait-mediated effects of natural
         enemies are closely connected to each other and jointly
         determine the effect of natural enemies on prey popula
        tion dynamics.
    '''
]

In [19]:
for abstract in abstracts_basic:
    sentences = abstract.split(".")
    for sentence in sentences:
        sentence_stripped = sentence.strip()
        relations = find_relations(sentence_stripped)

        print(f"Sentence: \"{sentence_stripped}\"")
        if not relations:
            print("No Relations")
        for r in relations:
            print(f"Subject: \"{r['subject'].text}\"")
            print(f"Subject Context: \"{find_context(sentence_stripped, r['subject'])}\"\n")
            print(f"Verb: \"{r['verb'].text}\"\n")
            print(f"Object: \"{r['object'].text}\"")
            print(f"Object Context: \"{find_context(sentence_stripped, r['object'])}\"")
        print()

03/22/2025 14:37:37 - INFO - 	 missing_keys: []
03/22/2025 14:37:37 - INFO - 	 unexpected_keys: []
03/22/2025 14:37:37 - INFO - 	 mismatched_keys: []
03/22/2025 14:37:37 - INFO - 	 error_msgs: []
03/22/2025 14:37:37 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
03/22/2025 14:37:37 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

03/22/2025 14:37:37 - INFO - 	 ***** Running Inference on 1 texts *****
03/22/2025 14:37:41 - INFO - 	 missing_keys: []
03/22/2025 14:37:41 - INFO - 	 unexpected_keys: []
03/22/2025 14:37:41 - INFO - 	 mismatched_keys: []
03/22/2025 14:37:41 - INFO - 	 error_msgs: []
03/22/2025 14:37:41 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
03/22/2025 14:37:41 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

03/22/2025 14:37:42 - INFO - 	 ***** Running Inference on 1 texts *****


...

Sentence: "Both SPECIESHOVERFLY and SPECIESLACEWING
         caused a significant increase in the percentage of TRAIT among offspring compared to control treatments,
         emphasising that wing induction in the presence of natural
         enemies is a general response in SPECIESAPHIDS"
No Relations

Sentence: "The
         percentage of winged offspring was, however, dependent
         on the TRAIT, with a small effect of predation
         on SPECIESAPHID TRAIT at very high and very low
         predation rates, and a strong response of SPECIESAPHIDS at
         medium predation rates"
No Relations



03/22/2025 14:54:47 - INFO - 	 missing_keys: []
03/22/2025 14:54:47 - INFO - 	 unexpected_keys: []
03/22/2025 14:54:47 - INFO - 	 mismatched_keys: []
03/22/2025 14:54:47 - INFO - 	 error_msgs: []
03/22/2025 14:54:47 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
03/22/2025 14:54:47 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

03/22/2025 14:54:48 - INFO - 	 ***** Running Inference on 1 texts *****


Sentence: "Aphid wing induction was
         influenced by the interplay between predation rate and
         the resultant prey density"
No Relations

Sentence: "Our results suggests that
         density-mediated and trait-mediated effects of natural
         enemies are closely connected to each other and jointly
         determine the effect of natural enemies on prey popula
        tion dynamics"
No Relations

Sentence: ""
No Relations

